In [175]:
import os

In [176]:
import pandas as pd

In [177]:
import folium

In [178]:
import geopandas as gpd

In [179]:
import webbrowser

In [180]:
import geojson

In [181]:
import json

In [182]:
from IPython.display import display

In [183]:
with open('countries.geojson', 'r') as f:
    data = geojson.load(f)

In [184]:
df = gpd.GeoDataFrame.from_features(data)

In [185]:
m = folium.Map(location=[51.05, 5.86],zoom_start=3)

In [186]:
countriesLastfm = pd.read_csv('countries_cut_two.csv', sep = ';',encoding='latin-1')
countriesLastfm = countriesLastfm.drop(columns = ['COUNTRY'], axis = 1)
topArtists = pd.read_csv('CompleteArtists.csv', sep = ';')
topArtists = topArtists.drop(columns = ['Unnamed: 0'])
topArtists.head()
countriesLastfm.head()

,NAME,GEONAME
0,Taiwan,Taiwan
1,Afghanistan,Afghanistan
2,Albania,Albania
3,Algeria,Algeria
4,American Samoa,American Samoa


In [214]:
Geoname = [None] * (topArtists['Country'].count())
zmena = countriesLastfm['NAME'] != countriesLastfm['GEONAME']
countriesLastfm[zmena]
dct = {}
for name, name2 in zip(countriesLastfm['NAME'], countriesLastfm['GEONAME']):
    dct[name] = name2
 



In [189]:
topArtists = topArtists.replace(dct)

Editing some of the names in Geopandas dataframe due to different country naming conventions: create a dictionary.

In [190]:
rankNad10 = topArtists['Rank'] <= 10
rankNad20 = topArtists['Rank'] <= 20

In [191]:
top10Artists = topArtists[rankNad10]
top20Artists = topArtists[rankNad20]

In [192]:
top10Artists.head()

,Country,Rank,Artist,Scrobbles
0,Taiwan,1,Coldplay,5486719
1,Taiwan,2,Adele,2939052
2,Taiwan,3,Ed Sheeran,1770632
3,Taiwan,4,Sia,2214287
4,Taiwan,5,Maroon 5,3609286


In [193]:
top10Artists[top10Artists['Country']=='Taiwan']

,Country,Rank,Artist,Scrobbles
0,Taiwan,1,Coldplay,5486719
1,Taiwan,2,Adele,2939052
2,Taiwan,3,Ed Sheeran,1770632
3,Taiwan,4,Sia,2214287
4,Taiwan,5,Maroon 5,3609286
5,Taiwan,6,Radiohead,4821070
6,Taiwan,7,Ariana Grande,1238796
7,Taiwan,8,The Chainsmokers,889190
8,Taiwan,9,Calvin Harris,2292220
9,Taiwan,10,The Weeknd,1401285


In [194]:
df.head()

,geometry,ADMIN,ISO_A3
0,"POLYGON ((-69.99694 12.57758, -69.93639 12.531...",Aruba,ABW
1,"POLYGON ((71.04980 38.40866, 71.05714 38.40903...",Afghanistan,AFG
2,"MULTIPOLYGON (((11.73752 -16.69258, 11.73851 -...",Angola,AGO
3,"MULTIPOLYGON (((-63.03767 18.21296, -63.09952 ...",Anguilla,AIA
4,"POLYGON ((19.74777 42.57890, 19.74601 42.57993...",Albania,ALB


In [211]:
cols = ['Top ' + str(i+1) for i in range(10)]
artists = ['Artist ' + str(i+1) for i in range(10)]
for i in range(len(artists)):
    df['Top '+ str(i+1)] = str(None)

In [212]:
for cntry in top10Artists['Country']:
    for i in range(10):
        try:
            df['Top ' + str(i+1)][df['ADMIN']==cntry] = top10Artists[top10Artists['Country']==cntry][top10Artists['Rank']==i+1]['Artist'].item()
        except:
            pass

C:\Users\matej\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.
C:\Users\matej\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: `item` has been deprecated and will be removed in a future version
  after removing the cwd from sys.path.
C:\Users\matej\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [206]:
df.head()

,geometry,ADMIN,ISO_A3,Top 1,Top 2,Top 3,Top 4,Top 5,Top 6,Top 7,Top 8,Top 9,Top 10
0,"POLYGON ((-69.99694 12.57758, -69.93639 12.531...",Aruba,ABW,Beyoncé,Michael Jackson,Calvin Harris,Major Lazer,Pitbull,The Chainsmokers,Marvin Gaye,The Beatles,Kanye West,Rihanna
1,"POLYGON ((71.04980 38.40866, 71.05714 38.40903...",Afghanistan,AFG,The xx,The Beatles,Radiohead,Pink Floyd,Lana Del Rey,Metallica,Nirvana,Drake,blink-182,Katy Perry
2,"MULTIPOLYGON (((11.73752 -16.69258, 11.73851 -...",Angola,AGO,Lady Gaga,Rihanna,Sia,Ariana Grande,Beyoncé,Adele,The Weeknd,Justin Bieber,Lana Del Rey,Florence + the Machine
3,"MULTIPOLYGON (((-63.03767 18.21296, -63.09952 ...",Anguilla,AIA,Bon Iver,The Weeknd,Sia,Ellie Goulding,Post Malone,Selena Gomez,Justin Bieber,Shawn Mendes,Years & Years,Charlie Puth
4,"POLYGON ((19.74777 42.57890, 19.74601 42.57993...",Albania,ALB,The Weeknd,Radiohead,Coldplay,Daft Punk,David Bowie,The Beatles,The xx,Arctic Monkeys,Red Hot Chili Peppers,Queen


In [207]:
m = folium.Map(location=[51.05, 5.86], zoom_start=3)

In [208]:
folium.GeoJson(
    df[['ADMIN', 'geometry'] + cols].to_json(),
    show=True,
    tooltip=folium.features.GeoJsonTooltip(
        fields=['ADMIN'] + cols,
        aliases=['Country'] + cols)
).add_to(m)

Unfortunately the map cannot be displayed in Jupyter notebook, but with help of _webbrowser_ library, we can save the output as html file and open it in a new window.

In [209]:
m.save('Top10Artists.html')

In [210]:
webbrowser.open_new('Top10Artists.html')

True